# <h1 style='font-family:Lato,sans-serif;color:#ca5800;text-align:center;'> Friendly Machine Learning Model Training

<span style='font-family:Lato,sans-serif;'>author: <span style='color:#1696d2'>[@baiochi](#https://github.com/baiochi/)

<h3 style='font-family:Lato,sans-serif;color:#fdbf11;font-size: 1.2em;'> Load libraries and functions

In [1]:
# Libraries, functions and custom configurations
from config.custom import *
# Load CSS for custom Markdown
HTML(open('./config/custom.css', 'r').read())

Numpy: 1.20.3
Pandas: 1.3.4
Sklearn: 1.0.2
Matplotlib: 3.4.3
Seaborn: 0.11.2


<h3 style='font-family:Lato,sans-serif;color:#fdbf11;font-size: 1.5em;'>Analysis Setup

In [2]:
configuration = {
	# Setup file locations
	'df_path' : 		'./data/',
	'df_name' : 		'application_train.csv',
	'pickle_path' : 	'pickle_data/first_cycle/',
	# Select estimator
	'estimator' : 		RandomForestClassifier,
	# Select parameters
	'estimator_params' : {
						'criterion' : ['gini', 'entropy'],
						'n_estimators' : range(1, 501),
						'max_depth' : [1, 2]
						},
	# Select size of dataset to split
	'data_size' : 		5,
	# Select metric of evaluation
	'metric' : 			'roc_auc',
	# Select type of Scoring ['classification', 'regression', 'clustering']
	'metric_type' : 	'classification',
	# Select number of hyperopt iterations
	'max_evals' : 		2
}

In [3]:
# Assign new variables for a cleaner code
df_path = configuration['df_path']
df_name = configuration['df_name']
pickle_path = configuration['pickle_path']
estimator = configuration['estimator']
estimator_params = configuration['estimator_params']
data_size = configuration['data_size']
train_size, test_size = select_df_split_size(data_size)
metric = configuration['metric']
metric_type = configuration['metric_type']
max_evals = configuration['max_evals']

<h3 style='font-family:Lato,sans-serif;color:#fdbf11;font-size: 1.5em;'> Load Data

In [4]:
df = pd.read_csv(df_path + df_name)
print(f'{CYAN}Dataset info{WHITE}\nObservations: {df.shape[0]}\nFeatures: {df.shape[1]}')
df.head()

Dataset info
Observations: 246008
Features: 122


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,...,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,456162,0,Cash loans,F,N,N,0,112500.000,700830.000,22738.500,585000.000,Unaccompanied,Working,Incomplete higher,Single / not married,House / apartment,0.020,-8676,-813,-4163.000,-1363,NaN,1,1,1,1,0,0,Core staff,1.000,2,2,FRIDAY,17,0,0,0,1,1,0,Trade: type 2,NaN,0.699,0.171,0.062,0.030,0.976,0.674,0.005,0.000,...,0.062,0.030,0.976,0.678,0.005,0.000,0.103,0.167,0.042,0.000,0.051,0.052,0.000,0.000,reg oper account,block of flats,0.040,Block,No,0.000,0.000,0.000,0.000,-589.000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0.000,0.000,0.000,1.000
1,134978,0,Cash loans,F,N,N,0,90000.000,375322.500,14422.500,324000.000,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,0.025,-13583,-223,-3554.000,-3287,NaN,1,1,0,1,0,0,High skill tech staff,2.000,2,2,MONDAY,11,0,0,0,0,0,0,Business Entity Type 3,0.541,0.200,0.769,0.023,0.057,0.981,0.735,0.016,0.000,...,0.023,0.057,0.981,0.738,0.016,0.000,0.103,0.042,0.083,0.013,0.019,0.019,0.000,0.000,reg oper account,block of flats,0.016,Block,No,0.000,0.000,0.000,0.000,-1409.000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0.000,1.000,0.000,3.000
2,318952,0,Cash loans,M,Y,N,0,180000.000,544491.000,16047.000,454500.000,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.036,-13993,-6202,-7971.000,-4175,9.000,1,1,1,1,0,0,Managers,2.000,2,2,THURSDAY,15,0,0,0,0,0,0,Business Entity Type 1,NaN,0.705,0.626,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,1.000,1.000,1.000,-675.000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0.000,1.000,1.000,3.000
3,361264,0,Cash loans,F,N,Y,0,270000.000,814041.000,28971.000,679500.000,Unaccompanied,Pensioner,Secondary / secondary special,Married,House / apartment,0.046,-22425,365243,-11805.000,-1732,NaN,1,0,0,1,1,0,NaN,2.000,1,1,TUESDAY,9,0,0,0,0,0,0,XNA,NaN,0.725,0.811,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012,NaN,No,2.000,0.000,2.000,0.000,-1588.000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0.000,0.000,0.000,1.000,4.000
4,260639,0,Cash loans,F,N,Y,0,144000.000,675000.000,21906.00

<h3 style='font-family:Lato,sans-serif;color:#fdbf11;font-size: 1.5em;'> Define target, features and split data

In [5]:
X = df.drop(['TARGET', 'SK_ID_CURR'], axis=1)
y = df['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, 
													train_size=train_size, 
													test_size=test_size, 
													stratify=y, random_state=42)
numeric_features     = X_train.select_dtypes(include=np.number).columns.tolist()
categorical_features = X_train.select_dtypes(exclude=np.number).columns.tolist()
print(f'{CYAN}Train dataset size:{WHITE} {X_train.shape}')
print(f'{CYAN}Test dataset size:{WHITE} {X_test.shape}')

Train dataset size: (9840, 120)
Test dataset size: (2461, 120)


<h3 style='font-family:Lato,sans-serif;color:#fdbf11;font-size: 1.5em;'> Create Baseline Model

In [6]:
numeric_features_pipeline = Pipeline([
    ('impute_num', SimpleImputer(strategy='mean')),
    ('std', MinMaxScaler())
])
categorical_features_pipeline = Pipeline([
    ('impute_cat', SimpleImputer(strategy='constant', fill_value='unknow')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
pre_processing_pipeline = ColumnTransformer([
    ('numeric_transformer', numeric_features_pipeline, numeric_features),
    ('categorical_transformer', categorical_features_pipeline ,categorical_features)
])
pipeline = Pipeline([
    ('pre_processing', pre_processing_pipeline),
    ('rf', estimator(random_state=42))
]).fit(X_train, y_train)

print(f'Score for baseline model with {estimator.__name__}')
display_metrics(X_train, X_test, y_train, y_test, pipeline, metric_type)

Score for baseline model with RandomForestClassifier
Train metrics:
roc_auc_score 1.000
f1_score 1.000
Test metrics:
roc_auc_score 0.652
f1_score 0.010


<h3 style='font-family:Lato,sans-serif;color:#fdbf11;font-size: 1.5em;'> Hyper-parameter Optimization

In [7]:
# create hyper-parameter space
hps_space = create_hps_space(hps_params=estimator_params)
# create objective function
objective_function = create_fmin_function(estimator=estimator, 
										pp_pipeline=pre_processing_pipeline,
										X_train=X_train, y_train=y_train,
										metric=metric,
										hps_space=hps_space)
# run hyper-parameter search
hps_search = fmin(fn=objective_function, 
					space=hps_space, 
					algo=tpe.suggest, 
					trials=Trials(), 
					rstate=np.random.default_rng(42),
					max_evals=max_evals)

100%|██████████| 2/2 [00:10<00:00,  5.18s/trial, best loss: -0.696679371905777] 


<h3 style='font-family:Lato,sans-serif;color:#fdbf11;font-size: 1.5em;'> Metrics summary

In [8]:
# get results
hps_best_params = space_eval(hps_space, hps_search)
print(f'{CYAN}{estimator.__name__}{WHITE} best parameters:')
for key, value in hps_best_params.items():
        print(f'{GREEN}', key, f'{WHITE}', value)

# create new pipeline with the best parameters
hps_model = Pipeline([('pp', pre_processing_pipeline),
                ('est', estimator(**hps_best_params, random_state=42))]
        ).fit(X_train, y_train)

# print results
print(f'\nScore for best model with hyperopt:')
display_metrics(X_train, X_test, y_train, y_test, hps_model, metric_type)

RandomForestClassifier best parameters:
 criterion  gini
 max_depth  1
 n_estimators  423

Score for best model with hyperopt:
Train metrics:
roc_auc_score 0.731
f1_score 0.000
Test metrics:
roc_auc_score 0.726
f1_score 0.000


<h3 style='font-family:Lato,sans-serif;color:#fdbf11;font-size: 1.5em;'> Saving model with Pickle

In [9]:
# calculate how much of data was used in the model training
if data_size < 10:
    data_size = '0' + str(data_size)

# save file
with open(pickle_path + f'{estimator.__name__}_{data_size}_pct.pickle', 'wb') as file:
    pickle.dump(hps_model, file, protocol=pickle.HIGHEST_PROTOCOL)
    print(f'File saved on {os.path.realpath(file.name)}')

File saved on /Users/baiochi/OneDrive/Documents/GitHub/LetsCode/Modulo 5 - Machine Learning/projeto/pickle_data/first_cycle/RandomForestClassifier_05_pct.pickle
